# Bigram Markov Chain Model

Now you'll build a more complex Markov chain that uses the last _two_ words (or bigram) to predict the next word. Now your dict `chain` should map a _tuple_ of words to a list of words that appear after it. So for example, one entry of this dict might be

```
chain = {
    ("it", "is"): ["the", "the", "not", "a", "a", "not", "the"],
    ...
}
```

As before, you should also include tags that indicate the beginning and end of a song, as well as line breaks. That is, a tuple might contain tags like `"<START>"`, `"<END>"`, and `"<N>"`, in addition to regular words. So if the song starts with the line "Is this the real life?" and ends with the line "Nothing really matters to me.", you would have a dictionary that looks like
```
chain = {
    (None, "<START>"): ["Is", ...],
    ("<START>", "Is"): ["this", ...],
    ("Is", "this"): ["the", ...],
    ("this", "the"): ["real", ...],
    ("the", "real"): ["life?", ...],
    ("real", "life?"): ["<N>", ...],
    ("<N>", "Nothing"): ["really", ...],
    ("Nothing", "really"): ["matters", ...],
    ("really", "matters"): ["to", ...],
    ("matters", "to"): ["me.", ...],
    ("to", "me."): ["<END>", ...],
    ...
}
```

In [1]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """
    chain = {(None, "<START>"): []}
    for song in lyrics: ## song is a list of lines = one song's lyrics
        l_count = 0
        end_count = len(song)-1
        for line in song:
            words = line.split()
            #print(len(words))
            for idx,word in enumerate(words):
                if len(words) == 1:
                    continue
                if idx == 0: # First word:
                    if l_count == 0: ## -of the song
                        chain[(None, "<START>")].append(word)
                        if ("<START>", word) in chain:
                            chain[("<START>", word)].append(words[1])
                        else:
                            chain[("<START>", word)] = [words[1]]
                    elif l_count > 0: ## -of line
                        if ("<N>", word) in chain:
                            chain[("<N>", word)].append(words[1])
                        elif ("<N>", word) not in chain:
                            chain[("<N>", word)] = [words[1]]
                        if last_word in chain:
                            chain[last_word].append(word)
                        elif last_word not in chain:
                            chain[last_word] = [word]
                if idx == len(words)-1: # Last word
                    if l_count == end_count: # -of the song
                        if (words[-2], words[-1]) in chain:
                            chain[(words[-2], words[-1])].append("<END>")
                        else:
                            chain[(words[-2], words[-1])] = ["<END>"]
                    else: # -of the line
                        if (words[-2], words[-1]) in chain:
                            chain[(words[-2], words[-1])].append("<N>")
                        else:
                            chain[(words[-2], words[-1])] = ["<N>"]
                else: # All other words
                    if (words[idx-1], word) in chain:
                        chain[(words[idx-1], word)].append(words[idx+1])
                    else:
                        chain[(words[idx-1], word)] = [words[idx+1]]
                    
                    
            if len(words) > 1:
                last_word = (words[-1], "<N>")   
            l_count+=1

    return chain

In [2]:
# Load the pickled lyrics object that you created in Part 1.
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyrics)

# What words tend to start a song (i.e., what words follow the <START> tag?)
print(chain[(None, "<START>")][:20])

['Hold', '[Intro:', 'Buddah', '[Intro:', 'Respect', '[Hook', 'Cocaina,', 'Bentley', 'If', 'I', 'You', 'They', 'Purps', 'Hold', 'Mama', 'Pop', 'Tell', 'Buddah', 'They', 'Mama']


Now, let's generate new lyrics using the Markov chain you constructed above. To do this, we'll begin at the `(None, "<START>")` state and randomly sample a word from the list of first words. Then, we'll randomly sample each next word from the list of words that appeared after the current word in the training data. We will continue this until we reach the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

In [3]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain[(None, "<START>")]))
    
    # YOUR CODE HERE
    words.append(random.choice(chain[("<START>"), words[-1]]))
    while words[-1] != "<END>":
        words.append(random.choice(chain[(words[-2], words[-1])]))
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [4]:
print(generate_new_lyrics(chain))

Hold up, get right witcha) 
 Hold on 
 Now you see me have a drag race (skrt) 
 I'm a dog, woof (grrr) 
 Beat the pot, oh 
 Bad bitches walkin' out with bags at the store (bad) 
 Cookin' up dope with a hundred (thumb) 
 Thumbin' through a hundred thousand start snowin' with it (hundred) 
 Go grab the Margielas, right there with the white like mayonnaise 
 Pull up in a brown paper bag (ah) 
 That bullshit and cap you can leave it out (cap) 
 I just can't understand her 
 She gon' eat this molly like it's rice (eat it up) 
 Bitches call me Quavo Ratatouille 
 Run with that fire, aim at his eye 
 Bentley truck like a Chef 
 Count up the kitchen with a Uzi (dope) 
 My bitch is a five, my bitch (aye) 
 Take me a missile (aye) 
 Take out the bed 
 Uh oh, count up the door was a part of the plan, get millions is working 
 Brown paper bag (ah) 
 With them snipers, no binoculars, we looking for you in trouble (think about it) 
 Two cups of purple just to start up (pop it, pop it pop it) 
 I hea

### Grader's Comments

- 
- 

[This question is worth 20 points.]

In [5]:
# This cell should only be modified only by a grader.
scores = [None]